## 2. Data acquisition and cleaning



### 2.1. Data Source :

#### Import necessary Libraries

In [1]:
import requests
import pandas as pd
import numpy as np
from bs4 import BeautifulSoup as bs

In [39]:
import numpy as np # library to handle data in a vectorized manner

import pandas as pd # library for data analsysis
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

import json # library to handle JSON files

!conda install -c conda-forge geopy --yes # uncomment this line if you haven't completed the Foursquare API lab
from geopy.geocoders import Nominatim # convert an address into latitude and longitude values

import requests # library to handle requests
from pandas.io.json import json_normalize # tranform JSON file into a pandas dataframe

# Matplotlib and associated plotting modules
import matplotlib.cm as cm
import matplotlib.colors as colors

# import k-means from clustering stage
from sklearn.cluster import KMeans

!conda install -c conda-forge folium=0.5.0 --yes # uncomment this line if you haven't completed the Foursquare API lab
import folium # map rendering library

print('Libraries imported.')

Solving environment: done

# All requested packages already installed.

Solving environment: done

# All requested packages already installed.

Libraries imported.


#### 2.1.1.Data Table : Sofia Quarter and Postalcode 

In [94]:
url='http://www.guide-bulgaria.com/SW/sofia-city/stolichna/sofia?t=postcodes'
html = requests.get(url).text
soup =bs(html,'lxml')
mylist= soup.find('div',{'class':'nine columns txt'}).find_all('li')
my_data=[]
mylist=[x.text.strip() for x in mylist]
my_data.append(mylist)
my_data=np.array(my_data).T.tolist()
df = pd.DataFrame(my_data,columns=["Data"])
new= df["Data"].str.split(' : ', n = 1, expand = True) 
df["Quarter"]= new[0]
df["Postalcode"]= new[1]
df.drop(columns =["Data"], inplace = True) 
df["Quarter"]= df["Quarter"].str.split(' ',n=1,expand=True).get(1)
df


,Quarter,Postalcode
0,Bakston,1618
1,Banishora,1233
2,Beli brezi,1680
3,Benkovski,1278
4,Borovo,1680
5,Botunets,1870
6,Boyana,1616
7,Chelopechene,1853
8,Darvenitsa,1756
9,Dianabad,1172


##### Use Nominatim to get geo coordinate for each quater (Test one quater works)

In [53]:
address = 'Lyulin 2, Sofia, Bulgaria'

geolocator = Nominatim(user_agent="foursquare_agent")
location = geolocator.geocode(address)

latitude = location.latitude
longitude = location.longitude
print(latitude, longitude)

42.7256471 23.2501109


####  2.1.2.Get geocode for each quarter 

In [106]:
quartergeo=[]
latitude = []
longitude = []

for quarter in df["Quarter"]:
    try:
        inputAddress = '{},Sofia, Bulgaria'.format(quarter)
        geolocator = Nominatim(user_agent="foursquare_agent")
        location = geolocator.geocode(inputAddress,timeout=20)
        quartergeo.append(quarter)
        latitude.append(location.latitude)
        longitude.append(location.longitude)
    except:
        latitude.append('NA')
        longitude.append('NA')
        print('Error, skipping address...')

df_geocodes = pd.DataFrame({'Quartergeo':quartergeo, 'Latitude':latitude,'Longitude':longitude})
df_geocodes.head()

Error, skipping address...
Error, skipping address...
Error, skipping address...
Error, skipping address...
Error, skipping address...
Error, skipping address...


,Quartergeo,Latitude,Longitude
0,Bakston,42.6655,23.27
1,Banishora,42.7104,23.3101
2,Beli brezi,42.6683,23.5991
3,Benkovski,42.7396,23.3461
4,Borovo,42.6694,23.2864


In [74]:
df_geocodes.head()

,Quartergeo,Latitude,Longitude
0,Bakston,42.6655,23.27
1,Banishora,42.7104,23.3101
2,Beli brezi,42.6683,23.5991
3,Benkovski,42.7396,23.3461
4,Borovo,42.6694,23.2864


In [113]:
left=df
right=df_geocodes
result = pd.merge(left, right, how='left', left_on='Quarter', right_on='Quartergeo')
result.drop(['Quartergeo'],axis=1,inplace=True)
result.dropna(inplace=True)
result['Latitude']= pd.to_numeric(result['Latitude'], errors='coerce')
result['Longitude']= pd.to_numeric(result['Longitude'], errors='coerce')
result = result[result['Longitude'].notna()]
result.reset_index(drop=True, inplace=True)
result

,Quarter,Postalcode,Latitude,Longitude
0,Bakston,1618,42.665486,23.269969
1,Banishora,1233,42.710366,23.310076
2,Beli brezi,1680,42.668338,23.599139
3,Benkovski,1278,42.739596,23.346105
4,Borovo,1680,42.669376,23.286375
5,Botunets,1870,42.739428,23.511576
6,Boyana,1616,42.646214,23.266749
7,Chelopechene,1853,42.732674,23.472598
8,Darvenitsa,1756,42.653300,23.361832
9,Dianabad,1172,42.662874,23.347948


In [114]:
result.shape

(77, 4)

In [90]:
address = 'Sofia, Bulgaria'
geolocator = Nominatim(user_agent="on_explorer")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print('The geograpical coordinate of Sofia are {}, {}.'.format(latitude, longitude))

The geograpical coordinate of Sofia are 42.6978634, 23.3221789.


#### 2.1.3. Use Foursquare to get Sofia venue data

##### Define Foursquare Credentials and Version

In [95]:
CLIENT_ID = 'TKGWHMQGKH04YK4SERDJT0BAWNXEZAV5NGL5GACBCNDE4USU' # your Foursquare ID
CLIENT_SECRET = 'UEUZN2RRMSKQQOUM0AEYKH5VO022EPFDMMPAPGKO4ZLTG3ZH' # your Foursquare Secret
VERSION = '20180604'
LIMIT = 30
print('Your credentails:')
print('CLIENT_ID: ' + CLIENT_ID)
print('CLIENT_SECRET:' + CLIENT_SECRET)

Your credentails:
CLIENT_ID: TKGWHMQGKH04YK4SERDJT0BAWNXEZAV5NGL5GACBCNDE4USU
CLIENT_SECRET:UEUZN2RRMSKQQOUM0AEYKH5VO022EPFDMMPAPGKO4ZLTG3ZH


In [96]:
result.loc[0, 'Quarter']

'Bakston'

In [98]:
quarter_latitude = result.loc[0, 'Latitude'] # latitude value
quarter_longitude = result.loc[0, 'Longitude'] # longitude value

quarter_name = result.loc[0, 'Quarter'] # quarter name

print('Latitude and longitude values of {} are {}, {}.'.format(quarter_name, 
                                                               quarter_latitude, 
                                                               quarter_longitude))

Latitude and longitude values of Bakston are 42.6654864, 23.269969.


##### First, let's create the GET request URL.

In [99]:
radius=500
limit=100
url= 'http://api.foursquare.com/v2/venues/explore?client_id={}&client_secret={}&ll={},{}&v={}&radius={}&limit={}'.format(CLIENT_ID,
        CLIENT_SECRET,
        quarter_latitude,quarter_longitude,
        VERSION,
        radius,
        limit)

In [101]:
jresults = requests.get(url).json()

In [102]:
# function that extracts the category of the venue
def get_category_type(row):
    try:
        categories_list = row['categories']
    except:
        categories_list = row['venue.categories']
        
    if len(categories_list) == 0:
        return None
    else:
        return categories_list[0]['name']

In [103]:
venues = jresults['response']['groups'][0]['items']
    
nearby_venues = json_normalize(venues) # flatten JSON

# filter columns
filtered_columns = ['venue.name', 'venue.categories', 'venue.location.lat', 'venue.location.lng']
nearby_venues =nearby_venues.loc[:, filtered_columns]

# filter the category for each row
nearby_venues['venue.categories'] = nearby_venues.apply(get_category_type, axis=1)

# clean columns
nearby_venues.columns = [col.split(".")[-1] for col in nearby_venues.columns]

nearby_venues.head()

,name,categories,lat,lng
0,Аванти,Liquor Store,42.665564,23.272955
1,Млекарницата на Добрев 4,Cheese Shop,42.664005,23.266223
2,Ресторант Слънце,Restaurant,42.664546,23.266423
3,Сладкарница Сладки Идеи,Café,42.669091,23.269640
4,Dekora.Fashion,Clothing Store,42.663667,23.265996


In [109]:
def getNearbyVenues(names, latitudes, longitudes, radius=500):
    
    venues_list=[]
    for name, lat, lng in zip(names, latitudes, longitudes):
        print(name)
            
        # create the API request URL
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius, 
            limit)
            
        # make the GET request
        jresults = requests.get(url).json()["response"]['groups'][0]['items']
        
        # return only relevant information for each nearby venue
        venues_list.append([(
            name, 
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in jresults])
        
    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['Quarter', 
                  'Quarter Latitude', 
                  'Quarter Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
    
    return(nearby_venues)

In [115]:
Sofia_venues = getNearbyVenues(names=result['Quarter'],
                                   latitudes=result['Latitude'],
                                   longitudes=result['Longitude']
                                  )

Bakston
Banishora
Beli brezi
Benkovski
Borovo
Botunets
Boyana
Chelopechene
Darvenitsa
Dianabad
Dragalevtsi
Fakulteta
Filipovtsi
Geo Milev
Gorna Banya
Gorublyane
Gotse Delchev
Hadji Dimitar
Hipodruma
Hladilnika
Hristo Botev
Ivan Vazov
Izgrev
Iztok
Krasna Polyana
Krasno Selo
Kremikovtsi
Lagera
Levski
Lozenets
Lyulin 1
Lyulin 10
Lyulin 2
Lyulin 3
Lyulin 4
Lyulin 5
Lyulin 6
Lyulin 7
Lyulin 8
Lyulin 9
Malashevtsi
Malinova Dolina
Manastirski Livadi
Mladost 1
Mladost 2
Mladost 3
Mladost 4
Moderno predgradie
Motopista
Musagenitsa
Nadejda
Nadejda 3
Nadejda 6
Obelya
Obelya 2
Ovcha kupel
Ovcha kupel 2
Pavlovo
Poduyane
Poligona
Reduta
Republika
Seslavtsi
Simeonovo
Slatina
Stefan Karadja
Strelbishte
Studentski grad
Suhata Reka
Suhodol
Sveta Troitsa
Trebich
Yavorov
Zaharna fabrika
Zapaden Park
Zona B-18
Zona B-19


2

In [116]:
print(Sofia_venues.shape)
Sofia_venues.head()

(1418, 7)


,Quarter,Quarter Latitude,Quarter Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
0,Bakston,42.665486,23.269969,Аванти,42.665564,23.272955,Liquor Store
1,Bakston,42.665486,23.269969,Млекарницата на Добрев 4,42.664005,23.266223,Cheese Shop
2,Bakston,42.665486,23.269969,Ресторант Слънце,42.664546,23.266423,Restaurant
3,Bakston,42.665486,23.269969,Сладкарница Сладки Идеи,42.669091,23.269640,Café
4,Bakston,42.665486,23.269969,Dekora.Fashion,42.663667,23.265996,Clothing Store


#### 2.1.4. Use Foursquare to get Paris 11e  venue data

In [124]:
address = '110 avenue parmentier, 75011 paris'

geolocator = Nominatim(user_agent="foursquare_agent")
location = geolocator.geocode(address)

p_latitude = location.latitude
p_longitude = location.longitude
print(p_latitude, p_longitude)

48.8672018 2.3732749


In [125]:
radius=500
limit=100
url= 'http://api.foursquare.com/v2/venues/explore?client_id={}&client_secret={}&ll={},{}&v={}&radius={}&limit={}'.format(CLIENT_ID,
        CLIENT_SECRET,
        p_latitude,p_longitude,
        VERSION,
        radius,
        limit)
jresults = requests.get(url).json()

In [126]:
# function that extracts the category of the venue
def get_category_type(row):
    try:
        categories_list = row['categories']
    except:
        categories_list = row['venue.categories']
        
    if len(categories_list) == 0:
        return None
    else:
        return categories_list[0]['name']

In [128]:
venues = jresults['response']['groups'][0]['items']
    
paris11_venues = json_normalize(venues) # flatten JSON

# filter columns
filtered_columns = ['venue.name', 'venue.categories', 'venue.location.lat', 'venue.location.lng']
paris11_venues =paris11_venues.loc[:, filtered_columns]

# filter the category for each row
paris11_venues['venue.categories'] = paris11_venues.apply(get_category_type, axis=1)

# clean columns
paris11_venues.columns = [col.split(".")[-1] for col in paris11_venues.columns]

paris11_venues.shape

(100, 4)

In [129]:
paris11_venues.head()

,name,categories,lat,lng
0,Le 11ème Domaine,Wine Bar,48.866896,2.373664
1,Tannat,Restaurant,48.868529,2.371895
2,Pili-Pili,Bar,48.867029,2.374910
3,Krügen,Creperie,48.868263,2.373917
4,Le Tiki Lounge,Cocktail Bar,48.868317,2.371210
